In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchinfo import summary

class NN(nn.Module):
    def __init__(self, input_size, num_classes): #input_size 28*28 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(in_features= input_size, out_features= 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
       x = F.relu(self.fc1(x))
       
       x = self.fc2(x)
    #    print(x.shape)
       return x
model = NN(784, 10)
data = torch.randn(64, 784)
print(model(data).shape)
# print(summary(model(data), input_size = data.shape))




torch.Size([64, 10])


In [2]:
# Hyperparameters
input_size = 28*28
batch_size = 64
num_epochs= 1
no_classes = 10
learning_rate = 0.001

In [3]:
# Dataset load
train_data = dataset.MNIST(root = 'dataset/', train = True, download = True, transform= transforms.ToTensor())
train_loader = DataLoader(dataset= train_data, batch_size= batch_size, shuffle= True)
test_data = dataset.MNIST(root= 'dataset/', train = False, download= True, transform= transforms.ToTensor())
test_loader = DataLoader(dataset= test_data, batch_size= batch_size, shuffle= True)

In [4]:
# initialise the network
model = NN(input_size= input_size, num_classes= no_classes)
# Loss and optimisers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)


In [6]:
# train the network

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.reshape(data.shape[0], -1) # (64, 1*28*28)
        # forward pass
        y = model(data)
        loss = criterion(y, target)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()




In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from genetic import GeneticAlgorithm, Individual

X, y = train_loader
X_test, y_test = test_loader

# Define the hyperparameter search space
hyperparameters = {

    'learning_rate': np.logspace(-3, 0, 4)

}

# Define the fitness function
def fitness_function(individual):
    # Create a Gradient Boosted model with the individual's hyperparameters
    # initialise the network
    model = NN(input_size= input_size, num_classes= no_classes)
    # Loss and optimisers
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params = model.parameters(), lr = individual.genes['learning_rate'])

    # Fit the model to the training data
    for epoch in range(individual.genes['epoch']):
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1) # (64, 1*28*28)
            # forward pass
            y = model(data)
            loss = criterion(y, target)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent
            optimizer.step()

    # Predict the labels for the test data
    with torch.no_grad():
        for x,y in test_loader:
            scores = model(x)
            
    y_pred = model(X_test)
    # Calculate the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    # Return the accuracy as the fitness value
    return accuracy

# Create the initial population
population = [Individual(hyperparameters) for _ in range(100)]

# Create the genetic algorithm
ga = GeneticAlgorithm(population, fitness_function)

# Run the genetic algorithm for 50 generations
best_individual = ga.run(50)

# Print the best individual's hyperparameters and fitness value
print('Best Hyperparameters:', best_individual.genes)
print('Best Fitness:', best_individual.fitness)
